<a href="https://colab.research.google.com/github/schmellerl/gradient_flows_order_parameters_mechanics/blob/main/colab/Example2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
try:
    import google.colab  # noqa: F401
except ImportError:
    import ufl
    import dolfin
else:
    try:
        import ufl
        import dolfin
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
        import ufl
        import dolfin

In [2]:
filepath = './Example2/'

<h1>Gradient flows for coupling order parameters and mechanics</h1>

FENICS implementation of the examples from

Schmeller, L. & Peschka, D. (2022). arXiv:XXXX.YYYY


DOI: 

<h3>General coupled gradient flow evolution</h3>

For a state variable $q=(u,\psi)$ with displacement $u:\Omega\to\mathbb{R}^d$ and phase field $\psi:\Omega\to\mathbb{R}^N$ we consider the following free energy 
\begin{align}
    \mathscr{F}(q) 
    = & 
    \int_{{\Omega}}W_{\rm elast}(F_e,\psi){\rm d}x
    + \int_{{\Omega}}W_{\rm phase}(\psi,\nabla\psi,F)
    {\rm d}x
\end{align}

for deformation gradient $F=\mathbb{I}_d+\nabla u=F_e F_p$ and with given plastic strain $F_p=F_p(\psi)$ and $F_e=FF_p^{-1}$.

We consider a Neo-Hookean elastic energy density 
\begin{align}
   W_{\rm elast}(F_e,\psi) = \frac{G}{2}\left(\text{tr}(F_e^\top F_e - \mathbb{I}_d) -2\log(\det(F_e))\right)+
 \frac{K}{2}\big(\det(F_e)-H\big)^2  
\end{align}
with bulk modulus $G=G(\psi)$ and phase volume $H=H(\psi)$ and inverse compressibility $K\in\mathbb{R}$. The remaining part of the free energy is 
\begin{align}
    W_\text{phase}(\psi,\nabla\psi,F) &= \left[\frac{1}{2}\nabla\psi\cdot \sigma\nabla\psi + W_\text{entropy}(\psi,F)\right]\det(F)
\end{align}
with second-order tensor $\sigma=\sigma(\psi,F)\in\mathbb{R}^{d\times d}$. 
Together with a given dissipation potential $R(q,\dot{q})$, different parameters, double-well or Flory-Huggins-type entropy $W_\text{entropy}$ and possible constraints $C(q,\nabla q)=0\in\mathbb{R}^M$ added through the Lagrangian in terms of $q_\lambda=(q,\lambda)$ with the optional Lagrange multiplier $\lambda:\Omega\to\mathbb{R}^M$
\begin{align}
\mathscr{L}(q_\lambda)=\mathscr{F}(q)+\int_\Omega C(q,\nabla q)\cdot\lambda\,{\rm d}x
\end{align}
we consider the formal gradient flow evolution $\partial_t q =-\nabla_R \mathscr{F}(q)$, which we solve by a saddle-point problem generated by the minimization problem
\begin{align}
%\partial_t q =-\nabla_R \mathscr{L}(q)
%\quad\Leftrightarrow\quad \left(
  \min_{v_\lambda=(v,\hat{\lambda})} \Big[R(q,v)+\langle \mathrm{D}\mathscr{L}(q_\lambda),v_\lambda\rangle\Big]%\right)
\end{align}
solved by incremental minimization as described in more detail in the manuscript.

In [4]:
# put in a separate file
from IPython.display import HTML, display
def progress(value, max=100):
    return HTML("""
        Iterations: {value} of {max}
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

from fenics import *
from matplotlib import pyplot as plt
import numpy as np
import random
from mshr import *

from mpl_toolkits.axes_grid1 import make_axes_locatable

plt.rcParams.update({'font.size': 14})

# should not appear in public version
def output_mesh(mesh):
    File(filepath + "mesh.xml") << mesh

def output_solution(q,n,t=0):
    u, psi,*_ = q.split()

    File(filepath + "u"+str(n)+".xml") << project(q.sub(0),FunctionSpace(mesh,Vu)) 
    File(filepath + "psi"+str(n)+".xml") << project(q.sub(1),FunctionSpace(mesh,Vpsi))
    #File(filepath + "eta_psi"+str(n)+".xml") << project(q.sub(2),FunctionSpace(mesh,U))


# Example 2:

For the second example we consider a free boundary problem starting with a circular dist in the center $(0,0)$ with radius $1$ such we do not impose Dirichlet boundary conditions. We use one species $N=1$ and other constitutive relations / paramters

\begin{align}
G&=10\\
K&=10\\
H&= (1+\psi-1/2)\\
\sigma&=\varepsilon F^{-1}F^{-\top} \qquad \varepsilon=0.05\\
F_p&=\mathbb{I}_2 \qquad\qquad\quad  \text{no multiplicative coupling}\\
W_\text{entropy}&=\frac{C}{\varepsilon}\big(\psi \ln(\psi)+\chi\psi(1-\psi)+ (1-\psi)\ln(1-\psi)\big) \qquad  C = 1 \text{ and Flory Huggins parameter }\chi = 3\\
R(q,\nabla q)&=\text{Cahn-Hilliard-type }R_\psi\text{ with }\mu=1\\
C(q,\nabla q)&= 
\begin{pmatrix}
(J-(1+\psi))\\
\int u_x {\rm d}x\\
\int u_y {\rm d}x\\
\int (-y^Tu_x + x^Tu_y) {\rm d}x\\
\end{pmatrix}
\text{ with }  \, u=(u_x,u_y).
\end{align}

 

In [ ]:
# model parameters
N_T   = 1500           # time steps
G     = Constant(10.)  # elastic modulus
K     = Constant(10.)  # Comp
mu    = Constant(1)    # Cahn-Hilliard mobility
eps   = Constant(0.05) # phase field energy parameter
C     = Constant(1)    # kB*T/vg
chi   = Constant(3.0)  # FH-Parameter
invnu = Constant(1)     

circ    = Circle(Point(0, 0), 1)
mesh    = generate_mesh(circ, 32) 

# define the function spaces
Vu    = VectorElement("P", mesh.ufl_cell(), 1) # V_u   = displacements
Vpsi  = FiniteElement("P", mesh.ufl_cell(), 1) # V_psi = order-parameter(s)
Uu    = VectorElement("P", mesh.ufl_cell(), 1)
Upsi  = FiniteElement("P", mesh.ufl_cell(), 1) # U     = forces
VxU   = FunctionSpace(mesh, MixedElement([Vu,Vpsi,Uu,Upsi])) # tensor space of V x U

R       = FiniteElement("Real", mesh.ufl_cell(), 0)    # real space -> conditions
VxUxRM  = FunctionSpace(mesh, MixedElement([Vu,Vpsi,Uu,Upsi, R, R, R]))   
xx      = interpolate( Expression(('x[0]','x[1]'),degree=2), FunctionSpace(mesh, Vu))


In [6]:
# incremental minimization for coupled finite strain elasticity + phase field
def incremental_minimization(old_q, tau):
    
    q, dq  = Function(VxUxRM), TestFunction(VxUxRM)
    
    u, psi, eta_u, eta_psi, lambda1, lambda2, lambda3 = split(q) # current solution
    old_u,old_psi,*_ = split(old_q) # old solution
    du,dpsi,deta_u,deta_psi,*_  = split(dq) # test functions

    # define continuum mechanics / plasticity variables
    d       = psi.geometric_dimension()
    I       = Identity(d)
    F       = I + grad(u)         # deformation gradient
    J       = det(F)              # Jacobian strain
    gradpsi = inv(F).T*grad(psi)

    nx = Expression(("1.0","0.0"), degree = 2)
    ny = Expression(("0.0","1.0"), degree = 2)
    
    # define free energy F_free 
    W_elastic    = (G/2)*(tr(F.T*F - I) - 2*ln(J)) + K/2*(J-(1+psi-1/2))**2
    W_phase      = eps/2*inner(gradpsi,gradpsi)*J
    W_phase     += C/eps*(psi*ln(psi)+chi*psi*(1-psi) + (1-psi)*ln(1-psi) )*J 

    #c0           = (J-(1+psi)) 
    c1           = dot(nx,u)
    c2           = dot(ny,u)
    c3           = (dot(u,nx)*xx.sub(1) - dot(u,ny)*xx.sub(0))  

    F_free    = (W_elastic + W_phase)*dx
    L         = F_free +  c1*lambda1*dx + c2*lambda2*dx + c3*lambda3*dx 

    # backward Euler time derivative
    dot_psi = (psi-old_psi)/tau
    dot_u   = (u-old_u)/tau

    # add energy and M_psi operator
    Res  = derivative(L, q, dq) - inner(eta_psi,dpsi)*dx - inner(eta_u,du)*dx 
    
    # add bilinear form a_psi and M_psi*
    Res += inner(dot_psi, deta_psi)*dx + inner(mu*grad(eta_psi), grad(deta_psi))*dx 
    Res += inner(dot_u, deta_u)*dx + inner(invnu*eta_u,deta_u)*dx
    
    # solve nonlinear problem using old solution as initial guess
    q.assign(old_q)    
    parameters["form_compiler"]["cpp_optimize"] = True
    parameters["form_compiler"]["quadrature_degree"] = 5

    solve(Res == 0, q)
    
    E_free     = assemble(F_free)
    E_elastic  = assemble(W_elastic*dx)
    E_phase    = assemble(W_phase*dx)

    return q,E_free, E_elastic, E_phase

In [ ]:
old_q = interpolate(Expression(("0","0","0.1+0.8*(float)rand()","0","0", "0", "0", "0", "0"), degree = 2),VxUxRM) 

t             = 0
tau           = 1e-6
q,E_free,E_elastic,E_phase = incremental_minimization(old_q, tau) 
old_q.assign(q)


energies     = [[0,0.0,E_free, E_elastic, E_phase]]

output_mesh(mesh)
output_solution(q,0,t)

out   = display(progress(0, N_T), display_id=True)
for n in range( N_T) :
    if n<20:
      tau = 0.1/N_T
    else:
      tau = 0.1/N_T

    t += tau
    q,E_free,E_elastic,E_phase = incremental_minimization(old_q, tau)
    energies.append([n,t,E_free, E_elastic,E_phase])
    output_solution(q,n+1,t)
    out.update(progress( n+1 , N_T )) 
    old_q.assign(q)

E = list(zip(*energies))
np.save(filepath + "energies",E)